In [1]:
import torch
from matplotlib import pyplot as plt
import numpy as np

def generate_data(N):
    X = torch.randint(0,9,size=(N,10))
    num2s = torch.count_nonzero(X==2,dim=-1)
    num4s = torch.count_nonzero(X==4,dim=-1)

    labels= (num4s > num2s)

    return X, labels

X, y = generate_data(123)


In [16]:
embed_func = torch.nn.Embedding(10, embedding_dim=16)

In [17]:
embedX = embed_func(X)

In [48]:
embedX.shape

torch.Size([123, 10, 16])

In [54]:
query = torch.nn.Parameter(torch.randn(1,16)) # query embedding
key_func = torch.nn.Linear(16, 16)

query.shape

torch.Size([1, 16])

In [50]:
keys = key_func(embedX)
keys.shape

torch.Size([123, 10, 16])

In [43]:
qk = torch.einsum('ie,bje->bij', query, keys)
qk = qk / 16**0.5
att = torch.nn.functional.softmax(qk, dim=-1)

#qk.sum()

In [33]:
class AttentionModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.query = torch.nn.Parameter(torch.randn(1,16)) # query embedding
        self.key_func = torch.nn.Linear(16, 16)
        self.embed_func = torch.nn.Embedding(10, embedding_dim=16)

        

        self.value_func = torch.nn.Sequential(
        torch.nn.Linear(16, 32),
        torch.nn.ReLU(),
        torch.nn.Linear(32, 1)
    )

        self.head_mlp = torch.nn.Sequential(
            torch.nn.Linear(1, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 1),
            torch.nn.Sigmoid()
        )

    def forward(self, X):
        embedX = self.embed_func(X)
        keys = self.key_func(embedX)
        qk = torch.einsum('ie,bje->bij', self.query, keys)
        qk = qk / 16**0.5
        att = torch.nn.functional.softmax(qk, dim=-1)

        vals = self.value_func(embedX)
        summary = torch.einsum('bij, bje->bie', att, vals)[:,0,:]
        pred = self.head_mlp(summary)

        
        return pred, att, vals
    

In [35]:
model = AttentionModel()

_,_,_ = model(X)



In [36]:
model = AttentionModel()

optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

X, y = generate_data(123)

for i in range(1000):
    
    pred, att, vals = model(X[0])
    loss = torch.nn.functional.binary_cross_entropy(pred, y.float())
    if i % 100 == 0:
        print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



RuntimeError: einsum(): the number of subscripts in the equation (3) does not match the number of dimensions (2) for operand 1 and no ellipsis was given

In [37]:
vals = value_func(embedX)

In [46]:
summary = torch.einsum('bij, bje->bie', att, vals)[:,0,:]

In [53]:
pred = head_mlp(summary)